# Objetivo de este notebook

#En este notebook se desarrollará todo el código necesario para establecer un modelo de recomendación de películas a partir la muestra de películas de Netflix conforme a las notas de referencia base:
#https://heuristic-bhabha-ae33da.netlify.app/sistemas-de-recomendaci%C3%B3n-y-filtrado-colaborativo.html#ejemplo-datos-de-netflix

In [1]:
# Requisitos

#Para correr las funciones de la librería libmf de python de este notebook, 
#debes clonar el repo de https://github.com/PorkShoulderHolder/python-libmf.git
#subir este notebook dentro de la carpeta /libmf/tests/

#El clonar estos archivos es necesario porque contienen funciones programadas o
#ejemplos construidos para exhibir el uso de estos paquetes

#si queres correr este notebook a través de docker con la imagen de
#jupyer numerical, debes instalar los paquetes libmf y colorama,
#ejecutando los comandos de abajo, reiniciando el kernel (luego
#puedes comentar con "#" los "pip install" para que no se vuelvan a ejecutar)

#Se deben descargar y colocar en la ruta donde está ubicado este notebook todos los archivos
#que se encuentran en este drive:
#https://drive.google.com/open?id=1JDWVCH5aCmRW9dLKiht80u5u1sgodgQ8

#A continuación se presentan las características de las películas, con el objeto
#de conocer un poco más los datos utilizados (pero esta no será la base usada para el modelo)

In [2]:
#pip install libmf

In [3]:
#pip install colorama

In [4]:
#import sys
#import numpy as np
#import types
#import scipy
#from scipy.sparse import random
#from scipy import stats
import pandas as pd
#import copy
#import random
#from functools import partial
#from colorama import init

#import libmf
#from libmf import mf

# Funciones auxiliares
Para explicar y ejemplificar el uso de libmf

In [14]:
#Función auxiliar para obtención de muestras aletorias
def muestra_val_ent(df,prop):
    """
    Separa al azar una base dada en 2 bases acorde una proporción de elementos
    dada
    
    params: df    pd.DataFrame (con 3 columnas=['usuario_id','peli_id',
                  'calif_x'])
            prop  float (0 < prop < 1) proporción de id_usuarios y
                  proporción de id_peliculas que se seleccionarán
    
    return: data_test      pd.DataFrame que tiene base que se usará
                           para prueba (por construcción el numero de
                           elementos de esta base no necesariamente coincide
                           con prop*total de elementos de df)
            data_training  pd.DataFrame que tiene base que se usará
                           para entrenamiento
    """
    #Base de todos los usuarios
    usuarios=pd.DataFrame(df['usuario_id'].unique(), columns=['usuario_id'])
    #Muestreo de usuarios
    valida_usuarios=usuarios.sample(frac=prop)  

    #Base de todas las películas
    peliculas=pd.DataFrame(df['peli_id'].unique(), columns=['peli_id'])
    #Muestreo de películas
    valida_pelis=peliculas.sample(frac=prop)
    
    #df_valida tiene las columnas usuario_id, peli_id y calif, pero
    #sólo incluye los usuarios muestreados en valida_usuarios
    df_valida=pd.merge(df,valida_usuarios, on ='usuario_id', how='inner')
    #data_valida filtra de la base df_valida los pelis_id muestreas
    #en valida_pelis
    data_test=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
    left=pd.merge(df, data_test, how='left',on=['usuario_id', 'peli_id'])
    data_training=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]
    return data_test, data_training

# Obtención de muestra base netflix

In [1]:
pwd

'/datos/proyecto-final-equipo5-mno-2020-1/Implementation/Implementacion_final_base_muestra_Netflix'

In [13]:
#ruta instancia
%cd /datos/proyecto-final-equipo5-mno-2020-1/Implementation/Codigo_Exploratorio_LIBMF/Sampling_Design/Reference_Files/

pelis_nombres=pd.read_csv('movies_title_fix.csv',names=["peli_id", "año", "nombre"])
pelis_nombres.head(10) #para ver cómo se encuentran los datos

/datos/proyecto-final-equipo5-mno-2020-1/Implementation/Codigo_Exploratorio_LIBMF/Sampling_Design/Reference_Files


,peli_id,año,nombre
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
5,6,1997.0,Sick
6,7,1992.0,8 Man
7,8,2004.0,What the #$*! Do We Know!?
8,9,1991.0,Class of Nuke 'Em High 2
9,10,2001.0,Fighter


In [17]:
pwd

'/datos/proyecto-final-equipo5-mno-2020-1/Implementation/Codigo_Exploratorio_LIBMF/Sampling_Design/Reference_Files'

In [16]:
#La base tiene peli_id, usuario_id_orig, calif, fecha y usuario_id. Se trabajará solo
#con las columnas de usuario, pelicula y calificación

%cd datos/datasets/dat_muestra_nflix.csv #RUTA DE BASE COMPLETA
df_muestra_netflix=pd.read_csv('dat_muestra_nflix.csv')
df_muestra_netflix_libmf=df_muestra_netflix[['usuario_id', 'peli_id', 'calif']]
df_muestra_netflix_libmf.shape

[Errno 2] No such file or directory: 'datos/datasets/dat_muestra_nflix.csv #RUTA DE BASE COMPLETA'
/datos/proyecto-final-equipo5-mno-2020-1/Implementation/Codigo_Exploratorio_LIBMF/Sampling_Design/Reference_Files


FileNotFoundError: [Errno 2] File dat_muestra_nflix.csv does not exist: 'dat_muestra_nflix.csv'

In [46]:
#Se genera una base de validación que contendrá el 20% de los datos
#y una base de entrenamiento que tendrá el restante 80%
random.seed(28882)

data_valida, data_entrena=muestra_val_ent(df_muestra_netflix_libmf,0.2)

################################################################################################################
#################### ESTO GUARDA LAS BASES QUE USA ESTE NOTEBOOK EN LA CARPETA DONDE ########################
################################# ESTA GUARDADO ESTE NOTEBOOK ###############################################
################################################################################################################
#puedes guardar las bases como un .csv dentro del directorio donde está este notebook
%cd /datos/datasets
data_valida.to_csv(r'data_valida.csv', index = False)    
data_entrena.to_csv(r'data_entrena.csv', index = False)    

#Base de todos los usuarios
usuarios=pd.DataFrame(df_muestra_netflix_libmf['usuario_id'].unique(), columns=['usuario_id'])
#Muestreo de usuarios
valida_usuarios=usuarios.sample(frac=0.2)  

#Base de todas las películas
peliculas=pd.DataFrame(df_muestra_netflix_libmf['peli_id'].unique(), columns=['peli_id'])
#Muestreo de películas
valida_pelis=peliculas.sample(frac=0.2)

/datos/datasets


In [47]:
#df_valida tiene las columnas usuario_id, peli_id y calif, pero
#sólo incluye los usuarios muestreados en valida_usuarios
df_valida=pd.merge(df_muestra_netflix_libmf,valida_usuarios, on ='usuario_id', how='inner')
#data_valida filtra de la base df_valida los pelis_id muestreas
#en valida_pelis
data_valida=pd.merge(df_valida,valida_pelis, on ='peli_id', how='inner')
#guardamos data_valida como un .csv dentro del directorio donde está este notebook
data_valida.to_csv(r'muestra_valida.csv', index = False)
left= pd.merge(df_muestra_netflix_libmf, data_valida, how='left',on=['usuario_id', 'peli_id'])
data_entrena=left[left.isnull().any(axis=1)][['usuario_id', 'peli_id','calif_x']]

In [48]:
display(data_valida.head(5))
data_valida.shape

,usuario_id,peli_id,calif
0,3,14686,4
1,125,14686,3
2,2036,14686,3
3,2729,14686,3
4,4054,14686,3


(855270, 3)

In [49]:
display(data_entrena.head(5))
data_entrena.shape

,usuario_id,peli_id,calif_x
0,1,1,3
1,2,1,3
2,3,1,4
3,4,1,4
4,5,1,4


(20113671, 3)

In [55]:
#podemos cambiar el directorio para que se guarde la base
#dentro del directorio donde están las funciones de libmf
#se guarda con la extensión .tr.txt porque es la extensión
#que requiere la paquetería de libmf

################################################################################################################
################# CON ESTE CÓDIGO GUARDA LAS BASES CREADAS DE data_valida.te.txt Y #############################
################### data_entrena.tr.txt REQUERIDAS PARA CORRER UN MODELO EN LIBMF ##############################
############ PERO IGUAL ESTO QUEDA DESECHADO SI SÓLO SE VA A OCUPAR LA LIBRERÍA LIBMF-PYTHON ###################
################################################################################################################
%cd /datos/miuser/libmf/demo
np.savetxt('data_valida.te.txt', data_valida.to_numpy(), delimiter=' ', fmt="%.0f")
np.savetxt('data_entrena.tr.txt', data_entrena.to_numpy(), delimiter=' ', fmt="%.0f")

/home/miuser/libmf/demo


In [56]:
#La base de df_muestra_valida_libmf ha quedado dividida por
#dos dataframes exluyenntes: data_valida y data_entrena
print("La suma de elementos de data_valida + data_entrena:",data_valida.shape[0] + data_entrena.shape[0])
print("El total de elementos de df_muestra_netflix_libmf es:",df_muestra_netflix_libmf.shape[0])

La suma de elementos de data_valida + data_entrena: 20968941
El total de elementos de df_muestra_netflix_libmf es: 20968941


In [57]:
modelo_base_ref(data_valida.to_numpy())

,row,col,mean_score_by_row,mean_score_by_col,total_score_mean
row,,,,,
4,4,128,4.207792,2.954545,3.609339
4,4,187,4.207792,3.078089,3.609339
4,4,571,4.207792,3.953481,3.609339
4,4,586,4.207792,3.247706,3.609339
4,4,708,4.207792,3.371808,3.609339
...,...,...,...,...,...
99912,99912,17307,5.000000,4.521127,3.609339
99928,99928,16169,2.000000,3.131050,3.609339
99937,99937,17293,3.000000,2.980172,3.609339


In [ ]:
################################################################################################################
###################### ESTO CORRE UN MODELO SIMPLE CON LIBMF A PARTIR DE LA BASE ###############################
############################### CREADAS ARRIBA, PERO ESTO YA NO SE CORRIÓ ######################################
################################################################################################################

#Pruebese los diferentes modelos ejemplo con data_entrena y data_valida
#En general estos algoritmos corren muy rápido
!/home/miuser/libmf/mf-train data_entrena.tr.txt model

# Funciones auxiliares
Para explicar y ejemplificar el uso de libmf

In [7]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def perform(fun, *args):
    """
    Evalúa una función con los argumentos indicados
    
    params: fun         Una función
            *args       Parámetros que utiliza la función "fun"
                        (se pueden meter cuantos parámetros tenga 
                        dicha funcion)
    return: fun(*args)  Valor de fun con parámetros *args
    """
    return fun(*args)

In [8]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def nan_or_value(x, f, *args):
    """
    Devuelve el valor de una función evaluada con sus parámetros
    *args acorde a la variable indicadora x
    
    params: x         boolean, si es 1 tómese valor de la función
                      si no será nan o 0 según especifique nan_zero
            fun       Una función
            *args     Parámetros que utiliza la función "fun"
                      (se pueden meter cuantos parámetros tenga 
                      dicha funcion)
            nan_zero  Si no se especifica nan devuelve 0
    return: fun(*args)  Valor de fun con parámetros *args
    """
    if(x==1):
        return perform(f,*args)
    else:
        return np.nan

In [9]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)

#Con esta vectorización se podrá aplicar a todos los elementos
#de un np.array la función nan_or_value
nan_or_value_vec=np.vectorize(nan_or_value, otypes=[object])

In [10]:
#Función para generar matrices ralas (sólo se usa para un ejemplo)
#Por su forma de construcción esta función es bastante ineficiente para
#ejemplos grandes
def crear_matriz_rala(n_ren,m_col,prop_elem_difer_nan,func_rand,*args_func_rand):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán nan cumpliendo que la proporción de éstos es de 
    1-prop_elem_difer_nan, donde los numeros diferentes de nan serán aleatorios
    generados a partir de una funcion aletorea que se envíe como parámetro
    (y sus argumentos)
    
    params: n_ren               int numero de renglones que tiene la matriz
            m_col               int numero de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
            func_rand           Función que genera un sólo no. aleatorio de
                                tipo float (no arreglos)
            *args_fun_rand      Argumentos necesarios para ejecutar la
                                la funcoión fun_rand
            nan_zero            Si no se especifica nan devuelve 0
    return: nan_or_value_vec    np.ndarray que contiene matriz rala
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    #se juntan 2 arreglos, uno de ceros y otro de unos, para después
    #reacomodarlos al azar
    M=np.concatenate((np.zeros(tot-k).astype(int), np.ones(k).astype(int)))
    M=np.array(random.sample(list(M),tot))
    M=M.reshape(n_ren,m_col)
    #Los unos se intercambias por numeros aleatorios y los ceros
    #por nan (o se dejan como cero)
    M=nan_or_value_vec(M,func_rand,*args_func_rand).astype(float)
    return M

In [11]:
#Función auziliar para generar matrices ralas (sólo se usa para un ejemplo)
def compactar_matriz_rala(M):
    """
    Pasa una matriz rala a su expresión compacta
    
    params: M  np.ndarray que representa matriz rala
    return: MC np.ndarray matriz en expresión compacta
               (no se reportarán los elementos con valor
               nan)
    """
    if type(M) is not np.ndarray:
        sys.exit('M debe ser de tipo numpy.ndarray')
    elif np.isnan(M).sum()==len(M):
        sys.exit('Todos los elementos de M son nan')
    MC=np.zeros((M[np.isnan(M)==False].size,3))
    k = 0
    n,m=M.shape
    for i in range(n): 
        for j in range(m): 
            if (~np.isnan(M[i,j])): 
                MC[k,0] = i
                MC[k,1] = j
                MC[k,2] = M[i,j] 
                k += 1
    return MC

In [13]:
#Función para generar matrices ralas en su expresión compacta
#Utilizada para varios ejemplos
def crear_matriz_rala_compacta(n_ren, m_col, prop_elem_difer_nan):
    """
    Construye matriz rala de n_renxm_col, seleccionando al azar los elementos
    que serán distintos de nan. Éstos elementos distintos de nan serán
    numero aleatorios entre 1 y 5 (uniforme discreta). Se dice compacta porque
    es una matriz solo señala las coordenadas o entredas de la matriz que 
    son diferentes a nan
    
    params: n_ren               int numero de renglones que tiene la matriz
            m_col               int numero de columnas que tiene la matriz
            prop_elem_difer_nan float (0 < prop_elem_difer_nan < 1) indica
                                la proporción de elementos distintos de nan
    return: M  np.ndarray que contiene matriz rala "compacta"
               (1era columna representa el índice de renglones,
               2da columna representa el índice de columnas,
               3era columna representa el valor del elemento
               o calificación en el contexto de netflix)
    """
    tot=int(n_ren*m_col)
    k=int(tot*prop_elem_difer_nan)
    M=random.sample(range(tot), k)
    columnas=np.mod(M,m_col)
    renglones=np.divide(M,m_col).astype(int)
    aleatorios=np.random.randint(1, 6, k)
    M=np.concatenate([renglones, columnas, aleatorios])
    M=M.reshape(3,k).T
    return M

In [13]:
#Función auxiliar al procesamiento de datos para la aplicación de un
#modelo de recomendación
def modelo_base_ref(MC):
    """
    Obtiene promedios por índices de renglón y columnas, así como del total
    (o calificaciones en el contexto de películas netflix)
    
    params: MC np.ndarray (con 3 columnasm, la 1era se toma como la
               de índices de renglon, la 2da como la de índices de columna
               y la 3era como el valor de los elementos o calificaciones en
               el contexto de netflix)
    return: count_by  pd.DataFrame de 3 columnas que tiene sólo los índices de
                      renglones o columnas que tienen al menos n elementos
    """
    df=pd.DataFrame(MC,columns=['row','col','score'])
    #Media total
    c=df['score'].mean(axis=0)

    #media por columna
    mean_by_col=df[['col','score']].groupby(['col']).mean()
    mean_by_col.rename(columns={'score':'mean_score_by_col'}, inplace=True)
    #media por renglon
    mean_by_row=df[['row','score']].groupby(['row']).mean()
    mean_by_row.rename(columns={'score':'mean_score_by_row'}, inplace=True)

    #se unen todas los resultados obtenidos, incluyendo la columna de
    #índice de columnas y renglones
    df=df.set_index('col').join(mean_by_col,how='left')
    df['col']=df.index
    df=df.set_index('row').join(mean_by_row,how='left')
    df['row']=df.index
    df=df[['row','col','mean_score_by_row','mean_score_by_col']]
    df['total_score_mean']=c
    
    return df

In [14]:
#Función auxiliar al procesamiento de datos para la aplicación de un
#modelo de recomendación
def filtrar(MC,ren_o_col,n):
    """
    Filtra los índices de renglon o columna que tengan más de n elementos
    (o calificaciones en el contexto de películas netflix)
    
    params: MC        np.ndarray (con 3 columnas, de nombre indistinto)
            ren_o_col string igual a 'row' o 'col'
            n         int no. elementos mínimos requeridos
    
    return: count_by  np.ndarray de 3 columnas que tiene sólo los índices de
                      renglones o columnas que tienen al menos n elementos
    """
    MC_df=pd.DataFrame(MC,columns=['row','col','score'])
    
    #este pequeño bloque calcula el total de elementos por columna o renglón
    count_by=MC_df[[ren_o_col,'score']].groupby([ren_o_col]).count()
    count_by.rename(columns={'score':'tot_counted'}, inplace=True)
    count_by[ren_o_col]=count_by.index
    count_by=count_by[[ren_o_col,'tot_counted']]
    
    #en este bloque se filtran los elementos de la matriz cuyo conteo por
    #columna o renglón tienen más de n elementos
    count_by=count_by[count_by.apply(lambda x: x['tot_counted'] >= n, axis=1)]
    count_by=MC_df.set_index(ren_o_col).join(count_by.set_index(ren_o_col),how='inner')
    count_by[ren_o_col]=count_by.index
    count_by=count_by[['row','col','score']]
    
    return count_by.to_numpy()

In [7]:
def imprime_atributos_MF(Obj):
    """
    Imprime los parámetros utilizados para factorización de la matriz contenida
    en Obj, mismos que son atributos del atributo _options
    
    params: Obj  libmf.mf.MF objeto de la librería libmf-python que contiene
                 la información requerida para hacer la factorización deseada
    
    return: None No regresa nada, sólo imprime los parámetros utilizados para
                 hacer la factorización
    """
    if type(Obj) is not libmf.mf.MF:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Atributos del modelo:')
    print('fun={0:d}'.format(Obj._options.fun))
    print('k={0:d}'.format(Obj._options.k))
    print('nr_threads={0:d}'.format(Obj._options.nr_threads))
    print('nr_bins={0:d}'.format(Obj._options.nr_bins))
    print('nr_iters={0:d}'.format(Obj._options.nr_iters))
    print('lambda_p1={0:0.4f}'.format(Obj._options.lambda_p1))
    print('lambda_p2={0:0.4f}'.format(Obj._options.lambda_p2))
    print('lambda_q1={0:0.4f}'.format(Obj._options.lambda_q1))
    print('lambda_q2={0:0.4f}'.format(Obj._options.lambda_q2))
    print('eta={0:0.4f}'.format(Obj._options.eta))
    print('do_nmf=',Obj._options.do_nmf)
    print('quiet=',Obj._options.quiet)
    print('copy_data=',Obj._options.copy_data)
    return None

In [35]:
def imprime_atributos_model(Obj):
    """
    Imprime los atributos del objeto model (definido dentro dentro de la
    librería libmf)
    
    params: Obj  libmf.mf.MFModel objeto de la librería libmf-python que
                 contiene los resultados obtenidos de la corrida del
                 modelo de factorización
                 
    return: None No regresa nada, sólo imprime los resultados contenidos
            en el objeto model, generado con los métodos del objeto MF
    """
    if type(Obj) is not libmf.mf.MFModel:
        sys.exit('A y b deben ser de tipo libmf.mf.MF')
    print('Resultados model:')
    print('fun={0:d}'.format(Obj.fun))
    print('m={0:d}'.format(Obj.m))
    print('n={0:d}'.format(Obj.n))
    print('k={0:d}'.format(Obj.m))
    print('b={0:0.4f}'.format(Obj.b))
    print('P={0:0.4f}'.format(Obj.P.contents.value))
    print('Q={0:0.4f}'.format(Obj.Q.contents.value))
    return None

In [18]:
#Elegiremos un formato de general para visualizar las matrices exhibidas como ejemplo
np.set_printoptions(sign=' ', precision=3)
#np.set_printoptions(sign=' ',formatter={'float': lambda x: "{0:0.3f}".format(x)})

# Matrices ralas (sparse)

In [19]:
#En este proyecto se exhibe el uso de las funciones de la librería libmf
#relacionadas a la factorización de matrices. El proyecto se enfoca al uso
#de estos algoritmos/métodos para los sistemas de recomendación de películas
#cuyo objetivo será obtener recomendaciones de películas para usuarios
#contenidos dentro de una base de datos proporcionada por netflix

#Si bien la base tiene datos como peli_id, usuario_id, calif, fecha, el
#algoritmo solo trabajará con los datos de calificaciones. Si bien existen
#diversos usuarios y películas, son pocas películas de las que se tienen
#calificaciones (escala del 1 al 5) proporcionadas por usuarios, por lo que
#la representación de estas calificaciones a través de una matriz (donde
#los usuarios serán los índices de los rengloens, las películas los índices
#de columnas y las calificaciones los valores de las entradas de la matriz),
#contendrá muchos huecos o calificaciones inexistentes,
#es decir, es una matriz rala (llena de ceros o valores no proporcionados)
#No obstante lo anterior, debido al numero de usuarios y películas
#existen muchos datos, y para no guardar todos los valores de cero (o NA)
#se opta por expresar y guardar dicha matriz en una forma compacta,
#la cual contiene por cada calificación el usuario que la generó y la película
#a la que corrresponde (es decir, es una matriz con 3 columnas)

# Procesos útiles para construcción de modelo
### Calificaciones promedio por usuario y/o película, y filtrado

In [46]:
#Para plantear un "modelo base de referencia", se calculan los medias
#por renglones (mean_ren) y por columnas (mean_col), así como 
#una media total (mean_tot), luego se sustituyen los valores de 
#la matriz (x_ij) por: mean_ren + mean_col - mean_tot
#La función modelo_base_ref regresa los promedios por renglon,
#columna y total. Con estos promedios se puede construir un modelo
#para heterogenizar el uso de la escala (calificaciones)

promedios=modelo_base_ref(MC)
display(promedios.head())
#podemos convertir a numpy array para realizar operaciones con
#variables de este tipo

#Recuerda pasa a np.array para hacer operaciones
promedios=promedios.to_numpy()
#el modelo base de ref quedaría así:
print('\n El modelo base de referencia es mean_ren + mean_col - mean_tot:') 
promedios[:,0]+promedios[:,1]-promedios[:,2]

NameError: name 'modelo_base_ref' is not defined

In [26]:
#En el camino para construir modelos adecuados, puede ser de interés
#o utilidad identificar los usuarios que tiene más películas calificadas
#o las películas con más calificaciones, más precisamente, que tengan
#más de "n" calificaciones.

#Usuarios que calificaron más de 2 películas
display(filtrar(MC,'row',2))
#Películas que fueron calificadas por más de 2 usuarios
display(filtrar(MC,'col',2))

array([[ 1,  2,  4],
       [ 1,  0,  1],
       [ 6,  3,  4],
       [ 6,  2,  2],
       [ 6,  0,  2],
       [10,  4,  5],
       [10,  3,  3],
       [14,  0,  5],
       [14,  3,  5],
       [15,  0,  1],
       [15,  2,  2],
       [16,  2,  1],
       [16,  1,  3],
       [17,  0,  2],
       [17,  1,  3]])

array([[17,  0,  2],
       [15,  0,  1],
       [ 1,  0,  1],
       [18,  0,  1],
       [14,  0,  5],
       [ 6,  0,  2],
       [ 3,  0,  3],
       [12,  1,  3],
       [ 7,  1,  4],
       [16,  1,  3],
       [17,  1,  3],
       [ 1,  2,  4],
       [15,  2,  2],
       [ 6,  2,  2],
       [ 5,  2,  1],
       [16,  2,  1],
       [13,  2,  5],
       [ 2,  2,  2],
       [ 6,  3,  4],
       [ 8,  3,  2],
       [10,  3,  3],
       [14,  3,  5],
       [10,  4,  5],
       [ 0,  4,  2],
       [ 4,  4,  1]])

In [327]:
#AQUÍ HAY QUE PONER LO QUE PUSO JAVI RESPECTO A FILTRAR LAS PELIS Y USUARIOS QUE APARECEN MUCHO

In [ ]:
#import sys
#!{sys.executable} -m pip install keras

In [ ]:
#import pandas as pd
#import numpy as np
#from keras.layers import Input, Embedding, Reshape, Dot, Concatenate, Dense, Dropout
#from keras.models import Model
#import random

In [ ]:
#Leemos el dataset 

In [ ]:
#df = pd.read_csv('~/jupyter/datasets/netflix_complete_cleaned.txt',sep=' ', header = None, names = ['Cust_Id','Movie_Id','Rating'], usecols = [0,1,2])

In [ ]:
#Hacemos un resumen 
print('Dataset shape: {}'.format(df.shape))
print('Dataset types:')
print(df.dtypes)
print('Summary statistics from dataset:')
df.describe()

In [ ]:
#Filtramos películas y usuarios con pocas evaluaciones para reducir el tamaño del dataset y mejorar predicciones. (Prune dataset)
#Filtramos deacuerdo al percentil 80: 
#Movie minimum times of review: 1799.0
#Customer minimum times of review: 52.0

In [ ]:
#f = ['count','mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.9),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

print('Movie minimum times of review: {}'.format(movie_benchmark))

#df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
#df_cust_summary.index = df_cust_summary.index.map(int)
#cust_benchmark = round(df_cust_summary['count'].quantile(0.9),0)
#drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

#print('Customer minimum times of review: {}'.format(cust_benchmark))

In [ ]:
#Se filtra el dataset original de acuerdo a los criterios de filtrado. 
# Filter sparse movies
min_movie_ratings = 1799
filter_movies = (df['Movie_Id'].value_counts()>min_movie_ratings)
filter_movies = filter_movies[filter_movies].index.tolist()

# Filter sparse users
min_user_ratings = 52
filter_users = (df['Cust_Id'].value_counts()>min_user_ratings)
filter_users = filter_users[filter_users].index.tolist()

# Actual filtering
df_filterd = df[(df['Movie_Id'].isin(filter_movies)) & (df['Cust_Id'].isin(filter_users))]
del filter_movies, filter_users, min_movie_ratings, min_user_ratings
print('Shape User-Ratings unfiltered:\t{}'.format(df.shape))
print('Shape User-Ratings filtered:\t{}'.format(df_filterd.shape))

In [ ]:
#Hacemos un shuffle del dataframe y lo dividimos en entrenamiento y prueba
random.seed(28882)
#random shuffle dataset
df2=df.sample(frac=1).reset_index(drop=True)
print(len(df2))

#dividimos dataset en train y test
n=round(len(df2)*.8) # Tamaño de muestra de entrenamiento (80% de los datos)
df_train=df2[-n:]
print(len(df_train)) # Número de valores en el dataset de entrenamiento
df_test=df2[:-n]
print(len(df_test)) # Número de valores en dataset de prueba

In [ ]:
#Imprimimos ejemplos del dataset de entrenamiento. 
df_train.to_csv(r'../datasets/netflix_train.txt',header=False,sep = ' ',index=False)
print('Full dataset shape: {}'.format(df_train.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

In [ ]:
#Imprimimos ejemplos del dataset de prueba. 

In [ ]:
df_test.to_csv(r'../datasets/netflix_test.txt',header=False,sep = ' ',index=False)
print('Full dataset shape: {}'.format(df_test.shape))
print('-Dataset examples-')
print(df.iloc[::5000000, :])

In [ ]:
#Insertar en libreria libmf

In [ ]:
!pwd

# Librería libmf python

In [35]:
# Python-libmf contiene de fondo el mismo código que el de la librería libmf, 
# aunque tiene programadas clases con atributos y métodos para poder ejecutar 
# los métodos de libmf. A continuación se señalan las principales 
# clases, atributos y métodos

# Clase: MF
# Atributos: .model
#            ._options 
#            .i
#            .j
# Métodos: .fit -> < Objetivo: factorize the i x j data matrix X into (j, k) (k, i) 
#                  <           sized matrices stored in MF.model
#                  < Parámetro X: (n, 3) shaped numpy array [known index and values 
#                  <              of the data matrix]
#
#          .predict -> < Objetivo: assuming we have already run the fit method, predict
#                      <           the values at certain indices of the data matrix
#                      < Parámetro X: (n, 2) shaped numpy array
#                      < Regresa: numpy array of length n
#
#          .mf_cross_validation -> < Objetivo: Realizar cross-validation
#                                  < Parámetro X: (n, 3)
#                                  < Parámetro folds: number of train / test splits
#                                  < Regresa: score across all folds 
#
#          .q_factors -> < Objetivo: Obtener la matriz Q
#
#          .p_factors -> < Objetivo:  Obtener la matriz P


# Clase: model
# Atributos: .fun
#            .m 
#            .n
#            .k
#            .b
#            .P
#            .Q
# Métodos: NA


# Clase: _options
# Atributos: .fun
#            .k
#            .nr_threads
#            .nr_bins
#            .nr_iters
#            .lambda_p1
#            .lambda_p2
#            .lambda_q1
#            .lambda_q2
#            .eta
#            .do_nmf
#            .quiet
#            .copy_data
# Métodos: NA

In [36]:
#A continuación un comparativo de los parámetros usados por python-libmf 
#y libmf, así como sus valores predeterminados

|Parámetro:|Nombre python-libmf|Valor por default python-libmf|Nombre libmf|Valor por default libmf|
|:--:|:--:|:--:|:--:|:--:|
|Función de pérdida|fun|0|-f|0|
|Numero de factores latentes|k|8|-k|8|
|Numero de threads|nr_threads|12|-s|12|
|Numero de bins|nr_bins|26|-n|adjusted by LIBMF for speed|
|Numero de iteraciones|nr_iters|20|-t|20|
|Parámetro de regularización L1 para P y Q<sup>(1)</sup>|lambda_p1|0.04|-l1|0|
|Parámetro de regularización L2 para P y Q<sup>(1)</sup>|lambda_p2|0|-l2|0.1|
|Parámetro de regularización L1 para P y Q<sup>(1)</sup>|lambda_q1|0.04|-l1|0|
|Parámetro de regularización L2 para P y Q<sup>(1)</sup>|lambda_q2|0|-l1|0.1|
|Tasa inicial de aprendizaje|eta|0.1|-r|0.1|
|Realizar factorización matricial no negativa<sup>(3)</sup>|do_nmf|False|--nmf|False|
|Sin outputs<sup>(3)</sup>|quite|False|--quite|False|
|Copiar datos|copy_data|True|NA|NA|
|Ruta conjunto validación|NA|NA|-p|ND|
|Numero de folds para validación cruzada<sup>(4)</sup>|ND|5|-v|1|
|Coeficiente de pérdida de entradas negativas|NA|NA|-a|1|
|Valor de entradas negativas<sup>(2)</sup>|NA|NA|-c|0.0001|
|Realizar entrenamiento a nivel de disco<sup>(3)</sup>|NA|NA|--disk|False|

<sup>(1)</sup> En libmf "l1" y "l2" se pueden definir con 1 o 2 parámetros, si se define uno se entiende que aplica tanto para P como Q, si se definen 2 el 1ero aplica para P y el 2do para Q

<sup>(2)</sup> Se supone que cada entrada positiva es 1

<sup>(3)</sup> En libmf su ausencia indica que no se llevará acabo. Por ejemplo, si no aparece "--nmf" en el llamado de mf-train, implica que las matrices P y Q pueden tener elementos negativos

<sup>(3)</sup> Para python-libmf si bien no es un parámetro que se defina cuando se crea el objeto MF como el resto de parámetros, se puede ejecutar una función de validación cruzada

In [20]:
# A continuación se corre un DEMO que muestra un poco el uso de estas funciones
# resalta que para programar este demo se definieron unas clases que a su vez
# ocupan las clases principales antes mencionadas de python-limbf
#ASEGURESE DE PONER LA UBICACIÓN CORRECTA DE LA UBICACIÓN DE LA CARPETA python-libmf

#Esto sólo corre sí te ubicas en la carpeta que contien mf_tests.py
# %cd /home/miuser/python-libmf/tests
%cd /datos/python-libmf/tests
!pwd
!python3 mf_tests.py

/datos/python-libmf/tests
/datos/python-libmf/tests
/home/miuser/.local/lib/python3.6/site-packages/libmf/__init__.py
Using file found in /home/miuser/.local/lib/python3.6/site-packages:
/home/miuser/.local/lib/python3.6/site-packages/libmf.cpython-36m-x86_64-linux-gnu.so
iter      tr_rmse          obj
   0       0.5077   3.1881e+02
   1       0.4834   2.9880e+02
   2       0.4385   2.6246e+02
   3       0.3795   2.1741e+02
   4       0.3299   1.8194e+02
   5       0.3009   1.6156e+02
   6       0.2876   1.5158e+02
   7       0.2790   1.4404e+02
   8       0.2751   1.3974e+02
   9       0.2733   1.3716e+02
  10       0.2723   1.3521e+02
  11       0.2727   1.3444e+02
  12       0.2710   1.3263e+02
  13       0.2717   1.3227e+02
  14       0.2707   1.3096e+02
  15       0.2715   1.3087e+02
  16       0.2708   1.3010e+02
  17       0.2710   1.2979e+02
  18       0.2706   1.2921e+02
  19       0.2706   1.2881e+02
testing cross val method
/home/miuser/.local/lib/python3.6/site-packages/lib

In [49]:
#se tiene una función para generar matrices ralas compactas sin embargo
#se observa que puede arrojar más de un valor para un mismo
#elemento de la matriz rala, vease abajo el renglón [0,1]
#tiene 3 valores asignados (0.166,0.217,0.556).

#El 1er parámetro señala indica que se seleccionará un valor
#de entre 0 y 19 para la 1er columna (que representa el índice
#de los renglones de la matriz rala que expresa); el 2do
#parámetro indica que se seleccionara al azar valores entre
#0 y 4 (que representa el índice de las columnas de la matriz
#rala que expresa); el 3er parámetro indica el número de elementos
#a generar 

#Fijaremos una semilla para reproducibilidad
np.random.seed(102)
MC=mf.generate_test_data(20, 5, 25)
print('Matriz generada con mf.generate_test_data(20,5,25):\n')
for i in range(MC.shape[0]):
    if MC[i,0]==0:
        print('\033[1;30;41m',MC[i,:])
    else:
        print('\033[0;0;0m',MC[i,:])
print('Tamaño MC:\n',MC.shape)

Matriz generada con mf.generate_test_data(20,5,25):

 [0.         1.         0.16637107]
 [19.          4.          0.30636304]
 [14.          3.          0.14721805]
 [18.          4.          0.22291025]
 [18.          1.          0.53951716]
 [15.          3.          0.74892651]
 [2.         2.         0.42831194]
 [11.          3.          0.10467911]
 [9.         3.         0.75337018]
 [8.         5.         0.59815912]
 [13.          0.          0.48677848]
 [8.         4.         0.97530286]
 [9.         2.         0.80230277]
 [15.          3.          0.33301479]
 [7.         5.         0.54855018]
 [4.         5.         0.31180599]
 [16.          4.          0.06230988]
 [6.         1.         0.90313856]
 [18.          0.          0.86054759]
 [3.        3.        0.6296915]
 [0.         1.         0.21655643]
 [0.        1.        0.5557169]
 [16.          5.          0.10932898]
 [6.         5.         0.55083566]
 [7.        0.        0.5812561]
Tamaño MC:
 (25, 3)


### Método mf.MF()
**CREAR OBJETO MF CON PARÁMETROS PARA EL MODELO DE FACTORIZACIÓN**

In [22]:
#Defínase un objeto de tipo MF para obtener la factorización deseada
mf_netflix=mf.MF()

#veamos algunas de las propiedades del objeto mf_netflix
print('Tipo de objeto: ',type(mf_netflix))
print('Instance variables: ',mf_netflix.__dict__)

Tipo de objeto:  <class 'libmf.mf.MF'>
Instance variables:  {'model': None, '_options': <libmf.mf.MFParam object at 0x7f9b94b0bd90>, 'i': None, 'j': None}


In [23]:
#Hasta el momento no se han definido los parámetros para correr el
#modelo de factorización, sin embargo por default se asignan algunos
#valores que a continuación se muestran:
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=8
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [24]:
#Observa que entre los atributos están los parámetros para que corra libmf
#y que definen el modelo (son los últimos 13; los que no les antecede y finalizas
#con "__")
print('Atributos: ', dir(mf_netflix._options))

Atributos:  ['__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'copy_data', 'do_nmf', 'eta', 'fun', 'k', 'lambda_p1', 'lambda_p2', 'lambda_q1', 'lambda_q2', 'nr_bins', 'nr_iters', 'nr_threads', 'quiet']


In [25]:
#Sin embargo podemos definir parámetros distintos pasando los argumentos
#Aquí lo hacemos en forma de kwargs, es decir, con el nombre de las variables
mf_netflix=mf.MF(fun=0,k=10,nr_threads=12,nr_bins=26,nr_iters=20,lambda_p1=0.04,lambda_p2=0.0,lambda_q1=0.04,lambda_q2=0.0,eta=0.1,do_nmf=False,quiet=False,copy_data=True)
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=10
nr_threads=12
nr_bins=26
nr_iters=20
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1000
do_nmf= False
quiet= False
copy_data= True


In [26]:
#Si optas por usar los parámetros por default excepto algunos
#puedes pasar los parámetros que quieres definir diferente
mf_netflix=mf.MF(eta=0.11,nr_iters=21, k=5)

#Nota que en este ejemplo como no definiste k=10, se inicializa con k=8
#pues es el valor por default
imprime_atributos_MF(mf_netflix)

Atributos del modelo:
fun=0
k=5
nr_threads=12
nr_bins=26
nr_iters=21
lambda_p1=0.0400
lambda_p2=0.0000
lambda_q1=0.0400
lambda_q2=0.0000
eta=0.1100
do_nmf= False
quiet= False
copy_data= True


**AJUSTE DEL MODELO DE FACTORIZACIÓN**

In [1]:
#Generamos una matriz rala al azar y la dividimos en entrenamiento y prueba
#para probar las funciones de factorización
np.random.seed(104)
random.seed(104)
MC=crear_matriz_rala_compacta(20,5,0.70)
print('El tamaño de MC es:',MC.shape)
MC_valida, MC_entrena=muestra_val_ent(pd.DataFrame(MC,columns=['usuario_id', 'peli_id', 'calif']),0.3)

MC_valida=MC_valida.to_numpy()
print('El tamaño de MC_valida es:',MC_valida.shape)
print('La matriz MC_valida queda así:\n',MC_valida)
MC_entrena=MC_entrena.to_numpy()
print('El tamaño de MC_entrena es:',MC_entrena.shape)

#IMPLEMENTAMOS MODELO CON MC_entrena
mf_netflix.fit(MC_entrena)

NameError: name 'np' is not defined

**VALIDACIÓN CRUZADA**

In [73]:
#Para obtener la validación cruzada basta con llamar al método, pudiendo
#usar más de 5 folds (que son los que tiene por default)
print('Validación cruzada con 5 folds: ',mf_netflix.mf_cross_validation(MC_entrena))
print('Validación cruzada con 10 folds: ',mf_netflix.mf_cross_validation(MC_entrena,folds=10))

Validación cruzada con 5 folds:  5.600449088459954
Validación cruzada con 10 folds:  5.65233440497207


**OBTENCIÓN DE PREDICCIONES**

In [76]:
#Para hacer la predicción se le pasa sólo los
#índices de los elementos a predecir, es decir
#una base con sólo 2 columnas que indican el 
#numero de renglon y columnas
pred=mf_netflix.predict(MC_valida[:,0:2])
pred

array([5.153527 , 4.825397 , 8.341425 , 4.825397 , 3.2621028, 4.825397 ,
       4.825397 ], dtype=float32)

In [77]:
print('Los valores verdaderos de MC_valida son:',MC_valida[:,2])

Los valores verdaderos de MC_valida son: [5 5 3 1 3 1 3]


In [78]:
print('Los resultados guardados en model están dispuestos como atributos: \n',dir(mf_netflix.model))

Los resultados guardados en model están dispuestos como atributos: 
 ['P', 'Q', '__class__', '__ctypes_from_outparam__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_b_base_', '_b_needsfree_', '_fields_', '_objects', 'b', 'fun', 'k', 'm', 'n']


In [88]:
imprime_atributos_model(mf_netflix.model)

Resultados model:
fun=0
m=20
n=20
k=20
b=4.8254
P=0.2524
Q=0.6674


**OBTENCIÓN MATRIZ Q**

In [80]:
Q=mf_netflix.q_factors()
print('Tamaño Q:',Q.shape)
print('Matriz Q:\n',Q)

Tamaño Q: (20, 5)
Matriz Q:
 [[ 0.6610217   1.3479118   1.0919917   1.2702276   1.1628327 ]
 [ 2.937817    1.0527891   0.03643222  2.3495717   1.5668904 ]
 [ 1.2376165   0.35759652  0.33090428  2.0263586  -0.20625529]
 [ 0.11610576  0.72134197  0.94133824  0.9837166   0.85301137]
 [ 2.3528585   1.0501823   0.23388717  0.41905427  1.3710599 ]
 [ 1.3444692   3.0200129   2.9119592  -1.0968567   1.9647042 ]
 [        nan         nan         nan         nan         nan]
 [ 0.08580175  2.0008779   1.2663869   1.4169183   1.1707867 ]
 [        nan         nan         nan         nan         nan]
 [ 0.6156498   1.2251899   0.7842371   0.9606924   1.1395668 ]
 [ 0.953333    0.45333335  0.64606535  2.293686    0.8497824 ]
 [ 1.4442893   0.795129    1.5840993   0.91383183  1.2142758 ]
 [ 1.81049     1.2567213   1.1931634   0.7990054   1.6075099 ]
 [        nan         nan         nan         nan         nan]
 [        nan         nan         nan         nan         nan]
 [ 0.46566537  1.1254746  

**OBTENCIÓN MATRIZ P**

In [84]:
PT=np.transpose(mf_netflix.p_factors())
print('Tamaño P:',PT.shape)
print('Matriz P:\n',P)

Tamaño P: (5, 20)
Matriz P:
 [[ 0.24917328  2.0498247   2.450214   -1.2872744   1.0924109 ]
 [ 0.39128682  0.339704    0.27196008  0.56796956  0.40583083]
 [-0.04710024  1.5104944   1.236175    1.1068976   0.6560206 ]
 [ 1.4916217   1.0703013   0.44259232  0.61636275  1.068625  ]
 [-0.41503382  0.96455663  0.8601089   0.64968723  0.11055897]
 [-0.30292937  0.5191632   1.1353921   2.7898161   0.45832276]
 [ 3.153519    1.9275347   1.0857233   1.2294562   2.2382514 ]
 [ 0.03649817  0.7632229   0.7629989   0.20383182  0.76700634]
 [ 0.3325359   0.5348337  -0.02616032  0.21619724 -0.03347047]
 [ 2.3174741   1.7705752   1.2476735   1.0245948   1.9362841 ]
 [ 0.7522779   0.56110036  1.7676197   2.0292437   2.6130407 ]
 [ 0.8474843   0.89459443  1.0397512   1.1229731   1.1285201 ]
 [ 2.4470503   1.686142    0.9568744   2.5065172   1.8143847 ]
 [        nan         nan         nan         nan         nan]
 [        nan         nan         nan         nan         nan]
 [        nan         nan 

**OBTENCIÓN MATRIZ R**

In [87]:
R=PT@Q
print('Tamaño R:',R.shape)
print('Matriz R:\n',R)

Tamaño R: (5, 5)
Matriz R:
 [[nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]
 [nan nan nan nan nan]]


In [37]:
R=P@np.transpose(Q)
print('Tamaño R:',R.shape)
print('Matriz R:\n',R)

Tamaño R: (20, 20)
Matriz R:
 [[ 5.199  1.652 -0.957  3.586  4.189 17.116    nan  6.711    nan  4.61
   0.835  6.048  6.646    nan    nan  2.671  5.613    nan  3.806  3.694]
 [ 2.256  3.552  1.802  1.485  2.188  2.547    nan  2.377    nan  1.912
   2.383  2.306  2.614    nan    nan  2.987  3.033    nan  2.311  1.484]
 [ 5.53   5.161  2.993  3.965  3.134  8.054    nan  6.9      nan  4.587
   4.597  4.892  5.231    nan    nan  7.325  6.57     nan  5.484  2.949]
 [ 4.945  8.639  3.434  2.879  6.507  7.98     nan  4.96     nan  4.398
   4.468  5.561  6.804    nan    nan  4.966  6.185    nan  5.056  3.978]
 [ 2.837  1.407  1.299  2.181  0.602  4.277    nan  3.938    nan  2.269
   2.116  2.179  2.107    nan    nan  4.124  3.032    nan  2.821  1.191]
 [ 5.823  6.876  5.662  4.585  1.844  2.176    nan  6.872    nan  4.473
   7.43   4.785  4.36     nan    nan 11.013  8.231    nan  6.102  2.319]
 [10.038 17.643  7.05   5.91  13.28  16.211    nan  9.868    nan  8.892
   9.225 11.607 13.985    nan

In [38]:
#Podemos ver cómo se ve la matriz R en forma compacta
compactar_matriz_rala(R)
pd.DataFrame(compactar_matriz_rala(R))

,0,1,2
0,0.0,0.0,5.198862
1,0.0,1.0,1.652449
2,0.0,2.0,-0.956831
3,0.0,3.0,3.586059
4,0.0,4.0,4.188980
...,...,...,...
220,19.0,12.0,6.406300
221,19.0,15.0,5.573576
222,19.0,16.0,7.153277
223,19.0,18.0,4.306216


In [39]:
#Notese que R justo contiene algunas de las estimaciones arrojadas por
#el método predict, pero no todas
comparativo=pd.DataFrame(columns=['ren','col','calif','R_calif_est','mf_netflix_predict'])
for i in range(MC_valida.shape[0]):
    comparativo.loc[i] = [MC_valida[i,0], MC_valida[i,1], MC_valida[i,2],R[MC_valida[i,0],MC_valida[i,1]],pred[i]]
display(comparativo)

#Observa que R no contiene todas las estimaciones que tiene predict
#checamos unos casos particulares: 4, 9, 11, 12
#nota que todos esos usuarios aparecen tanto en MC_entrena como 
#en MC_valida, y sin embargo la matriz R no tiene las calificaciones
#ni del usuario 13, 15 y 16
df_MC_valida=pd.DataFrame(MC_valida,columns=['ren','col','calif'])
df_MC_entrena=pd.DataFrame(MC_entrena,columns=['ren','col','calif'])
print('Usuarios 13, 15 y 16 en MC_valida:')
display(df_MC_valida[df_MC_valida.ren.isin([13,15,16])])
print('Usuarios 13, 15 y 16 en MC_entrena:')
display(df_MC_entrena[df_MC_entrena.ren.isin([13,15,16])])


#Obsérvese las películas que los usuarios 13, 15 y 16
#Tanto en entrena como en valida aparecen estos usuarios
#La base valida sólo tiene calificaciones de las películas
#3 y 4
print('Se calificaron películas 3 y 4 en MC_valida?...sí')
display(df_MC_valida[df_MC_valida.col.isin([3,4])])
print('Se calificaron películas 3 y 4 en MC_entrena?...sí')
display(df_MC_entrena[df_MC_entrena.col.isin([3,4])])


,ren,col,calif,R_calif_est,mf_netflix_predict
0,19.0,4.0,5.0,5.197598,5.197598
1,16.0,4.0,5.0,NaN,4.825397
2,17.0,4.0,3.0,8.488568,8.488568
3,15.0,4.0,1.0,NaN,4.825397
4,19.0,3.0,3.0,3.287389,3.287389
5,16.0,3.0,1.0,NaN,4.825397
6,13.0,3.0,3.0,NaN,4.825397


Usuarios 13, 15 y 16 en MC_valida:


,ren,col,calif
1,16,4,5
3,15,4,1
5,16,3,1
6,13,3,3


Usuarios 13, 15 y 16 en MC_entrena:


,ren,col,calif
32,15,2,1
34,16,1,5
40,15,0,3
56,13,2,2


Se calificaron películas 3 y 4 en MC_valida?...sí


,ren,col,calif
0,19,4,5
1,16,4,5
2,17,4,3
3,15,4,1
4,19,3,3
5,16,3,1
6,13,3,3


Se calificaron películas 3 y 4 en MC_entrena?...sí


,ren,col,calif
2,18,3,4
3,5,4,1
10,2,3,1
11,3,3,1
13,1,4,3
14,18,4,2
17,14,4,1
21,7,3,5
24,11,4,2
27,6,3,5
